# Motivation

This section will cover the motivation and idea behind choosing the topic of overfishing and explain which datasets are used and why.


## Problem statement (Goal for end user)

People are already aware of the general story on overfishing, even Netflix has jumped the bandwagon and produced documentaries such as _Seaspiracy_. However, such documentaries seem to mislead the audience [ref](https://www.theguardian.com/environment/2021/mar/31/seaspiracy-netflix-documentary-accused-of-misrepresentation-by-participants) due to over-dramatization. Moreover, most people think of fish as bad for the environment and our world sustainability goals, so they tend to avoid them all together, for example by becoming a vegetarian or vegan.


> What we aim to show, is a sound and objective analysis of overfishing. Providing insights into the best and worst ways of the fishing industry by analysing two particularly interesting countries: China and Norway. Finally, using our analysis, we discuss whether eating fish is a viable option for steady protein supply.



## Datasets

In terms of datasets, we use a wide variety of different datasets to compose our story.

### FAO

The **main** source of data is from _Food and Agriculture Organization of the United Nations_ at the url [https://www.fao.org/home/en](https://www.fao.org/home/en). The organization provides access to a massive database with anything related to food and agricultural data in the world.
Our requirements for the dataset was to have data for most countries (and further on, only Norway and China), some temporal information (years) and come from a credible source.
This yielded the datasets from FAO:

- Fishing industry (import, export, supply, production)
- Fish and seafood consumption per capita
- Sources of consumed protein
- Fishing methods

### Additional sources

Another source of data was the world bank [https://datacatalog.worldbank.org/search/dataset/0037712](https://datacatalog.worldbank.org/search/dataset/0037712)
This gave access to the trend and development of open-ocean fishing versus aquacultural fishing.

Additionally, we were able to get information on the GHG (Greenhouse Gas) emission of aquacultural fishing from [https://www.nature.com/articles/s41586-021-03889-2](https://www.nature.com/articles/s41586-021-03889-2)
### Descriptive data

Finally, we also made use of other third-party datasets that made it easier for us to gather information on a country's GDP and population each year.

In order to join this descriptive data with the main datasets, we had to find a common connection between them. Going by just the country name is risky as some countries go by more than one name and others simply have different spellings. Therefore, we used the country ISO-3 code that is _the_ standard for identifying countries. By merging this mapping onto any dataset using the country name (and making a few manual changes). We were able to get an ISO-3 standardized country code for each dataset that needed to be merged with other data.

Another addition to the descriptive data is the use of a **geojson** file that is used for the choropleth maps. However, the original file was too complex (too many polygons) and therefore took a lot of time to plot and render. We therefore decided to simplify the geojson using an online tool such that we ensure that we maintain interpretability and a high enough resolution for the polygons to overlap correctly with the country borders.

## Why did we choose these datasets

Choosing datasets for this project was not easy as it's difficult to find data that covers information on specific fishing related topics such as fishing gear, emission of aquacultural fish, protein supply for each country and so on. Even more so, it was painful to find data with a temporal aspect over multiple years. This resulted in us fetching data from many different sources after searching through a large amount of infeasible datasets.

We find that the selected datasets are useful because they are a combination of being rich in information (covers many metrics over many countries with few NaN values), covering multiple years of data, cover extremely niche or rare data.

The combination of the datasets provide us with an opportunity to explore overfishing on a global scope as well as diving into the specific methods of China and Norway. Moreover, they also contain information on eating habits in terms of protein supply which helps in analysing both the behaviour of the population and the demand of fish.

# Basic Stats

## Choices in data cleaning and preprocessing

## Dataset stats and key points

# Data Analysis

# Genre

## Which genre

## Visual Narrative

## Narrative Structure

# Visualizations

# Website

# Discussion

# Contributions